In [ ]:
# !pip install git+https://github.com/ourownstory/neural_prophet.git

In [ ]:
import matplotlib as mpl
mpl.rcParams["figure.dpi"] = 150
savefig_options = dict(format="png", bbox_inches="tight")

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

from neuralprophet import NeuralProphet

In [ ]:
INPUT_DATA_PATH = "https://raw.githubusercontent.com/MforMubashshera/forecast-water-quality/development-nms/data/"

label = "colifast_LAE"
df = pd.read_csv(
    INPUT_DATA_PATH + label + ".csv",
    parse_dates=["timestamp"]
)

df = pd.DataFrame({"ds": df["timestamp"], "y": df["colifast_LAE"]})

In [ ]:
# load data ata

df = df[["timestamp", "colifast_LAE"]]
df.rename(columns={"timestamp": "ds", "colifast_LAE": "y"}, inplace=True)

In [ ]:
# 24-steps ahead Neural Model with Long AR
model = NeuralProphet(
    n_forecasts=24,
    n_lags=7*24,
    learning_rate=0.01,
    num_hidden_layers=1,
    d_hidden=16,
)
model = model.highlight_nth_step_ahead_of_each_forecast(24)
metrics = model.fit(df, freq="H")

In [ ]:
future = model.make_future_dataframe(df, periods=3, n_historic_predictions=len(df))
forecast = model.predict(future)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
model.plot(forecast, xlabel="timestamp", ylabel="colifast_LAE", ax=ax)

ax.xaxis.label.set_size(28)
ax.yaxis.label.set_size(28)
ax.tick_params(axis="both", which="major", labelsize=24)
ax.set_title("colifast", fontsize=28, fontweight="bold")

fig.savefig("forecast_water_quality_cLAE.png", **savefig_options)

In [ ]:
fig_param = model.plot_parameters()
fig_param.savefig("parameters_cLAE.png", **savefig_options)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
ax.plot(metrics["MAE"], "ob", linewidth=6, label="Training Loss")  
ax.plot(metrics["MAE_val"], "-r", linewidth=2, label="Validation Loss")

ax.legend(loc="center right", fontsize=16)
ax.tick_params(axis="both", which="major", labelsize=20)
ax.set_xlabel("Epoch", fontsize=28, fontweight="bold")
ax.set_ylabel("Loss", fontsize=28, fontweight="bold")

ax.set_title("Model Loss (MAE)", fontsize=28, fontweight="bold")
fig.savefig("model_loss_mae_cLAE.png", **savefig_options)

---